# Setup

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Same model as before, but use GPU if available

In [2]:
torch.cuda.is_available()

False

In [3]:
device = torch.device("cuda:0")

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on the GPU")
else:
    device = torch.device("cpu")
    print("running on the CPU")

running on the CPU


In [6]:
torch.cuda.device_count()

0

In [7]:
# load training data
training_data = np.load("Data/training_data.npy", allow_pickle=True)

In [15]:
X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT =.1 #percentage of validation data
val_size = int(len(X)*VAL_PCT)
print(val_size)
print(X.shape)

2494
torch.Size([24946, 50, 50])


In [9]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5) #input, output, kernel size
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        # create some random data to figure out the number of the folowing fc layer
        x = torch.randn(50, 50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        
        #print(x[0].shape)
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2] #consider x as a batch and grab 1. element
        return x
    
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear) # flatten x after conv layers   
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
net = Net()

In [10]:
net.to(device)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [11]:
net = Net().to(device) # put on gpu if available

In [13]:
BATCH_SIZE = 100
EPOCHS = 1

def train(net):
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    loss_function = nn.MSELoss()

    for epoch in range(EPOCHS):
        #outputs = net(train_X.view(-1, 1, 50, 50))
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)): #go through batches
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50).to(device)
            batch_y = train_y[i:i+BATCH_SIZE].to(device)
        
            net.zero_grad()
            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()
    print(loss)